# Assignment 2.1 - Vehicle Routing Problem
JM0100-M-6 Business Analytics  
Myrthe Wouters  
u1273195

In [1]:
# Global imports
import pandas as pd
import numpy as np
from haversine import haversine, Unit
import math
import operator
from copy import deepcopy

In [2]:
# Load data
STORES = pd.read_excel('Data Excercise 2 - EMTE stores - BA 2020-1.xlsx', index_col='City Nr.')

In [3]:
STORES.head()

,Name,Address,Postal code,City,Lat,Long,Type
City Nr.,,,,,,,
0,EMTE HEADQUARTERS VEGHEL,CORRIDOR 11,5466RB,VEGHEL,51.606702,5.528046,NaN
1,EMTE ARKEL,DR H DE VRIESPLN 14,4241BW,ARKEL,51.864000,4.993040,Coop
2,EMTE ARNEMUIDEN FR,CLASINASTR 5,4341ER,ARNEMUIDEN,51.500010,3.677280,Jumbo
3,EMTE BATHMEN FR,LARENSEWG 18,7437BM,BATHMEN,52.249060,6.289990,Jumbo
4,EMTE BEEK EN DONK,HEUVELPLN 73,5741JJ,BEEK EN DONK,51.529300,5.632300,Jumbo


## Exercise 2.1

Write a construction heuristic to create a starting solution for the VRP based on the Insertion Method. 

### Calculate rounded distances between every two locations
I will use a matrix to store the distances between every two stores. This makes my algorithms more efficient, because I do not have to call the haversine formula every time I want to compute the distance between two stores.

In [4]:
def calc_dist(loc1, loc2):
    '''Defines rounded distance (km) between two locations'''
    coords1 = STORES.loc[loc1, 'Lat'], STORES.loc[loc1, 'Long']
    coords2 = STORES.loc[loc2, 'Lat'], STORES.loc[loc2, 'Long']
    dist = round(haversine(coords1, coords2))
    return dist

In [5]:
def dist_to_min(km, speed_kmh):
    """Defines the duration of a route in minutes"""
    speed_kmm = speed_kmh/60
    minutes = round(km/speed_kmm)
    return minutes

In [6]:
# Save all distance in global variable DIST_MATRIX and all travel_times in global variable TIME_MATRIX
DIST_MATRIX = np.zeros((len(STORES), len(STORES)))
TIME_MATRIX = np.zeros((len(STORES), len(STORES)))

for location_1 in STORES.index:
    for location_2 in STORES.index:
        dist = calc_dist(location_1, location_2)
        DIST_MATRIX[location_1, location_2] = dist
        TIME_MATRIX[location_1, location_2] = dist_to_min(dist, 90)

### Classes
For this assignment, I use object-oriented programming. For exercise 2.1, I define three classes:
* Location: class for stores with their metadata as attributes.
* Route: class for routes with their individual attributes and methods.
* RoutePlanner: class for entire schedules (i.e., a combination of routes that together visit every store exactly once) with attributes and methods to plan a schedule.

#### Location Class

In [7]:
class Location:
    
    def __init__(self, nr, name, lat, long, store_type):
        self.nr = nr
        self.name = name
        self.lat = lat
        self.long = long
        self.store_type = store_type
        self.visited = False
        self.distance_hq = DIST_MATRIX[0, nr]
    
    @property
    def is_jumbo(self):
        '''Boolean value that shows if location is a Jumbo location'''
        return True if self.store_type=='Jumbo' else False

In [8]:
def get_params(df, index):
    '''
    Function to get all necessary variables from STORES dataframe to instantiate a Location instance for every store
    '''
    params = df.iloc[index][['Name', 'Lat', 'Long', 'Type']].values
    name, lat, long, store_type = params
    return index, name, lat, long, store_type

#### Define Route Class

As mentioned before, I define a Route class for routes with their individual attributes and methods. 

*There are a few important attributes, properties and methods of the Route class to highlight:*

* the `farthest_store` attribute of a route is the location that is farthest a way from the HQ in that route
* the `inner_route` attribute of a route is a list of locations without start and end at HQ. 
* the `full_route` property of a route is a list of locations representing the entire route, including start and end at HQ. 
    * For example, if a route visits stores with numbers 4, 8 and 10 in that order. The `inner_route` of this route is `[4, 8, 10]`, while the `full_route` is `[0, 4, 8, 10, 0]`.
* the `all_hours_constraints` property defines if the route meets the constraints of John's working hours and the opening hours of the stores

In [9]:
class Route:
   
    # Set class variables
    max_working_mins = 11*60
    max_opening_mins = 8*60
    speed_kmh = 90
    
    def __init__(self, farthest_store, hq):
        self.inner_route=[] # inner_route is defined as the route without start and end at HQ
        self.farthest_store = farthest_store
        self.hq = hq
        self.is_jumbo = self.farthest_store.is_jumbo
    
    @property
    def meeting_time(self):
        '''Defines the meeting time at individiual stores in the route'''
        return 30 if self.is_jumbo else 20
    
    @property
    def full_route(self):
        '''Defines full route of a route, including the start and finish at headquarters'''
        full_route = self.inner_route.copy()
        full_route.insert(0, self.hq)
        full_route.append(self.hq)
        return full_route
    
    @property
    def total_distance(self):
        '''Defines total distance in km of the route'''
        return sum(self.distances(full_route=True))
    
    @property
    def cumsum(self):
        '''
        Defines cumulative sum of distances at each location. 
        We need this cumulative sum in order to convert the final schedule to an Excel file.
        '''
        cumsum = list(np.cumsum(self.distances(full_route=True)))
        cumsum.insert(0, 0)
        return cumsum
    
    @property
    def total_visit_time(self):
        '''Defines total time spend in all stores of route together'''
        return len(self.inner_route) * self.meeting_time
    
    @property
    def working_hours_constraint(self):
        '''
        Defines if route meets the constraint of John's working hours. John cannot work more than 11 hours = 11*60 minutes per day.
        Hence, this method checks if the total travel time of the route and the total visit time at the stores does not exceed 11 hours.
        '''
        return (self.travel_time(full_route=True) + self.total_visit_time) <= Route.max_working_mins
    
    @property
    def opening_hours_constraint(self):
        '''
        Defines if route meets the constraint of visiting hours 09:00-17:00 at every store in route.
        This means that traveling time from the first store to the last store in the route (i.e., the inner route) plus total visiting time
        at the stores does not exceed 8 hours.
        '''
        return (self.travel_time(full_route=False) + self.total_visit_time) <= Route.max_opening_mins
    
    @property
    def all_hours_constraints(self):
        '''Defines if route meets both constraints John's working hours and visiting hours at every store'''
        return self.working_hours_constraint and self.opening_hours_constraint

    def distances(self, full_route=False):
        '''
        Defines all distances between consecutive locations in a route.
        The argument full_route can be set to either True or False. False is default. If full_route is set to False, this method defines the 
        distances between consecutive locations in the inner route (i.e., excluding start and end at HQ). While if full_route is set to True, 
        this method defines the distances between consecutive locations in the full route (i.e., including start and end at HQ).
        '''
        if full_route:
            distances = [DIST_MATRIX[self.full_route[idx].nr, self.full_route[idx+1].nr] for 
                         idx, _ in enumerate(self.full_route[:-1])]
        else:    
            distances = [DIST_MATRIX[self.inner_route[idx].nr, self.inner_route[idx+1].nr] for 
                         idx, _ in enumerate(self.inner_route[:-1])]
        return distances
    
    def travel_time(self, full_route=False):
        '''
        Defines the total travel time of the route, i.e., the time spend travelling in driving to stores in route.
        The argument full_route can be set to either True or False. False is default. If full_route is set to False, this method defines the 
        travel time of the inner route (i.e., excluding start and end at HQ). While if full_route is set to True, this method defines the 
        travel time of the full route (i.e., including start and end at HQ).
        '''
        if full_route:
            travel_times = [TIME_MATRIX[self.full_route[idx].nr, self.full_route[idx+1].nr] for 
                            idx, _ in enumerate(self.full_route[:-1])]
        else:    
            travel_times = [TIME_MATRIX[self.inner_route[idx].nr, self.inner_route[idx+1].nr] for 
                            idx, _ in enumerate(self.inner_route[:-1])]
        return sum(travel_times)
    
    def insert(self, pos, item):
        '''Insert location at given position in a route'''
        self.inner_route.insert(pos, item)
    
    def remove(self, item):
        '''Remove location from a route'''
        self.inner_route.remove(item)

#### RoutePlanner class
This class includes attributes and methods to generate a schedule (i.e., starting solution for VRP) through using a construction heuristic based on the Insertion Method.

*The generation of a schedule through the RoutePlanner class is implemented in the following way:*

Start with $j=1$
1. Select the farthest unplanned store based on distance from headquarters and plan this store in a new route $r_j$ 
2. Sort all potential unplanned stores (i.e., of same type (Jumbo or Coop/other) as farthest store) in increasing distance from farthest store
3. Try to plan each potential unplanned store $s_i$ at every position in route $r_j$. If there are feasible positions for $s_i$, insert $s_i$ at the best feasible position (i.e., the one that leads to the lowest total distance of route $r_j$). If there is no feasible positions for $s_i$, skip $s_i$ for route $r_j$
4. As long as there are unplanned stores left to visit, $j=j+1$ and continue at step 1.

Step 1-3 for one iteration are implemented in the method `plan_route` and the iterative algorithm is implemented in the method `solve`.

*There are a few important attributes, properties and methods of the RoutePlanner class to highlight:*

* the `locations` attribute contains a list of all 133 locations to visit in the entire schedule
* the `hq` attribute represents the headquarters
* the `loc_distances_hq` property defines a sorted list of all locations in decreasing distance from headquarters
* the `current_farthest_store` property defines an unplanned store $s$ that is farthest away from the headquarters at the start of iteration j
* the `potential_stores` method defines potential stores to visit given a particular farthest store. Potential stores include unplanned stores with same store type as the farthest store
* the `dist_farthest_store` method defines a sorted list of potential stores based on distance from farthest store
* the `plan_route` method executes the construction heuristic for one iteration, i.e., defines one route
* the `solve` method executes the entire construction heuristic and creates the schedule 

In [10]:
class RoutePlanner:
    
    def __init__(self, locations):
        self.locations = locations[1:]
        self.hq = locations[0]
        
    @property
    def loc_distances_hq(self):
        '''Sorts stores in decreasing distance from headquarter'''
        sorted_dist = sorted(self.locations, key=lambda x: x.distance_hq, reverse=True)
        return sorted_dist
    
    @property
    def to_visit(self):
        '''Defines the list of stores that still need to be visited in a future route'''
        stores_to_visit = [loc for loc in self.locations if loc.visited==False]
        return stores_to_visit
    
    @property
    def current_farthest_store(self):
        '''Defines current farthest store from HQ that has not been visited yet'''
        farthest_store = [loc for loc in self.loc_distances_hq if loc.visited==False][0]
        return farthest_store
    
    @property
    def stores_left(self):
        '''Defines if there are any stores left to visit'''
        return len(self.to_visit)>0
    
    def potential_stores(self, farthest_store):
        '''
        Defines potential stores for a given route with given farthest location.
        Potential stores for a route can be seen as stores that have the same store type as the farthest store (Jumbo or Coop/other) and that
        have not been visited in an earlier route (on an earlier day).
        '''
        if farthest_store.is_jumbo:
            potential_stores = [loc for loc in self.to_visit if loc.store_type=='Jumbo' and loc!=farthest_store]
        else: 
            potential_stores = [loc for loc in self.to_visit if loc.store_type!= 'Jumbo' and loc!=farthest_store]
        return potential_stores

    def dist_farthest_store(self, farthest_store):
        '''Sorts all the remaining unplanned stores in increasing distance from the current farthest store'''
        distances = [(DIST_MATRIX[farthest_store.nr, potential_store.nr], potential_store) 
                     for potential_store in self.potential_stores(farthest_store)]
        distances.sort(key=operator.itemgetter(0))
        sorted_dist = [potential_store for dist, potential_store in distances]
        return sorted_dist
    
    def plan_route(self, route):
        '''Plans individual routes, starting at the current farthest store'''
        
        # Insert current farthest store to be visited in a route
        route.insert(0, route.farthest_store)
        
        # All potential locations in ascending order
        for i in self.dist_farthest_store(route.farthest_store):
            best_dist = math.inf
            best_pos = None
            
            # Try location on every position in route
            for pos in range(len(route.inner_route)+1):
                
                # Insert location at position
                route.insert(pos, i)
                
                # Check if location at given position meets constraints and;
                # Check if this position is better than all previous viable positions for this location
                if route.all_hours_constraints and route.total_distance<best_dist:
                    best_dist = route.total_distance
                    best_pos = pos
                
                # Remove location from route
                route.remove(i)
            
            # If there is a viable position, insert location at best viable position in route
            if best_pos!=None:
                route.insert(best_pos, i)
        
        # Set all locations that are inserted to route to visited
        for loc in route.inner_route:
            loc.visited = True
            
    def solve(self):
        '''Solves VRP for all locations, i.e., makes final schedule through construction heuristic based on Insertion Method'''
        
        # Set route number
        j = 0
        routes = {} # Empty dict to store schedule
        
        # While there are unplanned stores, we should create a new route
        while self.stores_left:
            j += 1 # Update route number
            farthest_store = self.current_farthest_store # Assign current farthest store
            route = Route(farthest_store, self.hq) # Instantiate route instance with current farthest store
            self.plan_route(route) # Plan this route
            routes[j] = route # Store planned route with route nr as key in dict
            
        return routes

### Define function to save a schedule to an Excel file

In [11]:
def to_Excel(planned_routes, save=True, file_name=None):
    '''Creates dataframe with needed information from solved VRP, saves to Excel file if needed'''
    
    # Dictionary to store data from all routes
    data = {'Route Nr.': [], 
            'City Nr.': [],
            'City Name': [], 
            'Total Distance in Route (km)': [],
            'Total Distance (km)': []}
    
    # Define total kilometers traveled at start of the route in previous routes
    total_km_at_start = 0
    
    # All routes in planned routes
    for nr, route in planned_routes.items():
        cumsum_loc = 0  # Define current cumulative sum location
        
        # Add data for location to dictionary
        # Because routes are instances of Route class in Exercise 1, but of RouteDLL class in Exercise 2,
        # We need to make an if-statement here
        for location in route.full_route:
            
            data['Route Nr.'].append(nr)
            data['City Nr.'].append(location.nr)
            data['City Name'].append(location.name)
            data['Total Distance in Route (km)'].append(route.cumsum[cumsum_loc])
            data['Total Distance (km)'].append(route.cumsum[cumsum_loc] + total_km_at_start)
                
            cumsum_loc += 1 # Update current cumulative sum location
        
        # Update total kilometers traveled at start of the route in previous routes
        total_km_at_start += route.cumsum[-1] 
    
    df = pd.DataFrame.from_dict(data) # Save data to DataFrame
    
    # If needed, save data to Excel file
    if save:
        df.to_excel(file_name, index=False)
            
    return df

### Solution 2.1

In [12]:
# Instantiate location instances for each store
locations = [Location(*get_params(STORES, index)) for index, _ in STORES.iterrows()]

# Instantiate planner instance
planner = RoutePlanner(locations)
planned_routes_1 = planner.solve() # Create initial solution for VRP

# Save results to Excel file
df = to_Excel(planned_routes_1, file_name='Ex2.1-1273195.xls')

print('The initial schedule solution has a distance of {} km and contains {} routes.'.format(
    sum([route.total_distance for route in planned_routes_1.values()]), len(planned_routes_1)))

The initial schedule solution has a distance of 3019.0 km and contains 11 routes.
